Cette page doit être lancée sur Kaggle, depuis la compétition pour avoir accès aux données. Sinon vous devez récupérer les données séparément.

# IREN - Competition "a la mano"
## Ships classification with custom neural network

### Team:
**Jean Fechter** - jean.fecther

**Thibault Boutet** - thibault.boutet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Resizing
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sn

We define a seed to have reproducible results.

In [ ]:
import tensorflow as tf

SEED = 0
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Data loading
Let's load, display and get some informations on the data which are in the kaggle input.

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data",
    labels='inferred',
    label_mode="categorical",
    image_size=(16, 24)
)

In [ ]:
# Get the class labels from the dataset
class_labels = dataset.class_names
print(f"Number of classes: {len(class_labels)}")
print(class_labels)

In [ ]:
# Display the first image for each class
plt.figure(figsize=(15, 15))
for images, labels in dataset.take(1):
    for i in range(10):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_labels[i])
        plt.axis("off")

Although it's impossible for a human to classify those ships due to the low resolution of the images, let's see how a neural network handles this task.


## Data pre-processing

### Data augmentation
Data augmentation is a technique that expands a training dataset by applying transformations like rotations, translations, and flipping to the existing data. It improves model generalization and robustness by exposing it to a wider range of variations. This reduces overfitting and enhances pattern recognition.

After testing different configuration of data augmentation, we have chosen to apply:
* Rescaling
* Horizontal flip

The other data augmentation techniques such as rotation or translation increase a lot the training time for unsignificative results.
### Train/Validation split
We will split the dataset into **two sets**:
* Train set (**80%** of the dataset): Part of the dataset used to train the model
* Validation set (**20%** of the dataset): Part of the dataset used to test the model's accuracy during the training

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data',
    target_size=(16, 24),
    batch_size=32,
    class_mode='categorical',
    seed=SEED,
    subset='training',
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data',
    target_size=(16, 24),
    batch_size=32,
    class_mode='categorical',
    seed=SEED,
    subset='validation',
)

### Model Creation

Now that the data is loaded, let's create our custom model.
We are going to need different layers in order to make it as efficient as possible :
* Resizing : changes the size of the image. In our case, we double it as part of the data augmentation
* Conv2D : we use a 3x3 kernel and relu activation function to extract the features from the image
* MaxPooling2D : downsamples the input by selecting the maximum value within each local region to extract the most important features
* Dropout : randomly deactivates certain neurons, preventing overfitting
* BatchNormalization : normalizes the batch (mean = 0, standard deviation = 1) to make the network converge faster
* Flatten : reshapes the output into a 1D array
* Dense : fully connected (each neuron is connected to every neuron from the previous layer)

After testing different optimizers, we chose to use **Adam** as it gave the best results. We modified the learning rate to start at 0.01 (see RecudeLrOnPlateau bellow for more explainations)

In [ ]:

model = tf.keras.models.Sequential([
    Resizing(32, 48, interpolation="bicubic"),
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(16, 24, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    BatchNormalization(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
print("Number of layers : ", len(model.layers))

### Model training
Now that our model is defined, we need to train it on our data.
To improve the convergence of the model, we will use **callbacks**.

### Callbacks
A callback is a function that performs tasks during the training of a model. We will use an **EarlyStopping** callback and an **ReduceLrOnPlateau** callback.
* EarlyStopping: This callback allows to stop the training of the model when the loss or the accuracy does not improve after a certain number of epoch. We have chosen to stop the training when the accuracy does not improve after 5 consecutive epochs
* RecudeLrOnPlateau: This callback is used to adjust dynamically the learning rate during the training. We have chosen to wait 1 epoch of no improvment to reduce the learning rate by a 0.2 factor. Note that the learning rate can't be smaller than 1e-4

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=2)

lr_callback = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=1, min_lr=0.0001)

model_history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[early_stopping, lr_callback])

## Results analysis
### Curves
Once the training is finished, we want to plot the **loss** and the **accuracy** on the training and validation sets.
These two indicators measure the performance of the model.
Here are the formula to compute these indicators:
* Accuracy = Number of correct predictions / Total number of predictions
* Loss (Cross-Entropy Loss) = -Σ(y_actual * log(y_pred))

In [ ]:
#Loss
plt.plot(model_history.history['loss'], label='Train')
plt.plot(model_history.history['val_loss'], label='Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Model Loss')
plt.legend(loc='upper right')

In [ ]:
#Accuracy
plt.plot(model_history.history['accuracy'], label='Train')
plt.plot(model_history.history['val_accuracy'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Model Accuracy')
plt.legend(loc='upper right')

The ideal scenario for both graphs would be if both curves on each graph followed the exact same variations.<br>
We can observe that until epoch 12, they do.<br>After that, the accuracy and loss on the validation set starts to stabilize, improving little by little.<br>Because the validation curve for the loss never increases significantly, and the validation curve for the acuracy never decreases significantly, after epoch 12, we know that our model is **not overfitting**.

### Confusion matrix
A confusion matrix is a table that evaluates a classification model's performance by comparing predicted and actual values for each class. It provides key metrics like accuracy, precision, and recall, helping to assess the model's strengths and weaknesses in classifying different categories.

In order to build a confusion matrix, we need to have a set to make a prediction on with our model. Let's take the validation set to do the prediction and construct the matrix.

In [ ]:
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(validation_generator.classes, y_pred)
plt.figure(figsize=(10, 8))
sn.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

We notice that among other things, our model struggles to differentiate containerships, cruisers and destroyers.

## Submission
This part is responsible of the loading and prediction of the given test data by the network.

At the end, a *.csv* file is generated with the network's predictions.

In [ ]:
X_test = np.load('/kaggle/input/navires-2023-la-mano/test.npy', allow_pickle=True)
X_test = X_test.astype('float32') / 255

In [ ]:
res = model.predict(X_test).argmax(axis=1)
df = pd.DataFrame({"Category":res})
df.to_csv("reco_nav.csv", index_label="Id")

In [ ]:
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'reco_nav.csv')

## Conclusion
In this notebook, we have used a **custom hand-made model** to classify ships into 10 classes.
<br>
We had to try many different elements and strategies in order to find the best combination of pre-processing and layers for our model.<br>
Our network reaches **over 80% of accurate predictions** on the given test data.

##### 